In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("seaborn")

In [105]:
class ConBacktester():
    '''
    Class for the vectorised backtesting of contrarian Strategies.
    '''
    
    def __init__(self, symbol, start, end, spread = None, tcost = None):
        '''
        Parameters
        ----------
        symbol: str
            The ticker symbol of the instrument that will be backtested.
        start: str
            The start date for imported data.
        end: str
            The end date for imported data.
        spread:
            The spread between Buy and Sell prices. If you do not know the tcost,
            input this parameter.
        tcost: float
            The proportional transaction/trading costs per trade. 
        
        Note: For intraday_pairs.csv, data starts at 2018-01-01, and ends at 2019-12-30.
              Symbols on the file are EURUSD, GBPUSD and EURAUD.
        '''
        self.symbol = symbol
        self.start = start
        self.end = end
        self.spread = spread
        self.results = None
        self.get_data()
        self.tc = tcost
        if self.tc is None:
            self.get_tcost()
    
    def __repr__(self):
        return f'ContrarianBacktester(ticker = {self.symbol}, start = {self.start}, end = {self.end}, transactioncost = {self.tc})'
    
    def get_data(self):
        '''
        Imports the data from a given source. Default is intraday_pairs.csv.
        '''
        #reading csv, then creating a new frame for one instrument
        raw = pd.read_csv('/Users/dejanmccreery/Documents/Documents - Dejan’s MacBook Air/Hagmann Algo Bootcamp/Part3_Materials/intraday_pairs.csv', parse_dates=['time'], index_col='time')
        raw = raw[self.symbol].to_frame().dropna()
        #slicing for the specified timestamps
        raw = raw.loc[self.start:self.end].copy()
        #formatting
        raw.rename(columns = {self.symbol: 'Price'}, inplace = True)
        #obtaining log returns
        raw['Returns'] = np.log(raw/raw.shift(1))
        self.data = raw
    
    def get_tcost(self):
        
        data = self.data.copy().dropna()
        halfspread = (self.spread*0.0001)/2
        #trans cost is proportional tc to mean price of instrument
        self.tc = halfspread / data['Price'].mean()
    
    def test_strategy(self, window = 1):
        '''
        Backtests the strategy.
        
        Parameters
        
        window: int
            The time window (number of bars) to be considered for the strategy.
        '''
        self.window = window
        data = self.data.copy().dropna()
        
        #np.sign tells us which direction the instrument returns are
        #making it negative is the idea of the contrarian strategy 
        #our position is thus 1 for buy, -1 for sell
        data['Position'] = -np.sign(data['Returns'].rolling(self.window).mean())
        #obtaining the returns
        data['Strategy Returns'] = data.Position.shift(1) * data['Returns']
        data.dropna(inplace=True)
        data['Cumulative Returns'] = data.Returns.cumsum().apply(np.exp)
        data['Cumulative Strat Returns'] = data['Strategy Returns'].cumsum().apply(np.exp)
        #trades tells us when we change our position, i.e. close our position and open another
        #by using diff we get the difference between 1 & -1, helpfully showing two trades
        #fillna as NaN still means no trades made
        data['Trades'] = data.Position.diff().fillna(0).abs()
        #calculating the return including trading costs
        data['Strategy Net Return'] = data['Strategy Returns'] - (data.Trades * self.tc)
        data['Cumulative Net Strat Return'] = data['Strategy Net Return'].cumsum().apply(np.exp)
        self.results = data
        
        performance = data['Cumulative Net Strat Return'].iloc[-1]
        outperform = performance - data['Cumulative Returns'].iloc[-1]
        return round(performance, 6), round(outperform, 6)
    
    def plot_results(self):
        '''
        Plots the performance of the trading strategy, and compares to the benchmark 
        i.e. if you had simply bought and held the instrument.
        '''
        if self.results is None:
            print("Run test_strategy() first.")
        else:
            title = f'{self.symbol} | Window = {self.window}, Transaction Cost = {self.tc}'
            self.results[['Cumulative Returns', 'Cumulative Strat Returns']].plot(title=title, figsize = (11,7))
   
    def optimise_parameter(self, windowrange):
        '''
        Finds the optimal strategy by finding the global maximum given the window range.
        
        Parameters
        
        windowrange: tuple
            Tuple of the form (start, end, step size), indicating the range of possible
            window values. To be plugged into a range function.
        '''
        
        windowlist = range(*windowrange)
        
        #testing window sizes
        results = []
        for window in windowlist:
            results.append(self.test_strategy(window)[0])
        
        best_performance = np.max(results) #best performance
        optimum = windowlist[np.argmax(results)] #the parameters of the best performance
        
        #run the optimal strat
        self.test_strategy(optimum)
        
        #create a df with the variety of results
        resultsframe = pd.DataFrame(data = {"Window": windowlist, "Performance": results})
        resultsframe['Performance'] = results
        self.results_overview = resultsframe
        
        return optimum, best_performance

In [106]:
tester = ConBacktester(symbol='EURUSD', start='2018-05-05', end='2019-05-05', spread=1.5)

In [107]:
tester.test_strategy()

(1.051042, 0.113228)

In [108]:
tester.optimise_parameter(windowrange=(1,100,1))

(54, 1.202305)